# Gradient Boosted Trees Notebook

In [11]:
# preset values
# sensor readings per gesture iteration:
reading_count = 30

## Installs

In [60]:
#! pip install micromlgen
#! pip install matplotlib
#! pip install xgboost
#! pip install altair
#! pip install altair_viewer
#! pip install scipy
#! pip install scikit-learn


## Imports

In [61]:
#plot imports
import altair as alt

#dataframe imports
import pandas as pd

#model selection, evaluation imports
from sklearn.model_selection import train_test_split
#model imports
import xgboost as xgb

#general tool impports
from IPython.display import display
import json



ModuleNotFoundError: No module named 'sklearn'

## Load the data

In [23]:
raw_sensor_data = pd.read_csv('raw_data_collection/dummy_data.txt', delimiter = '\t')
raw_sensor_data.head()

,iteration,gesture_code,sensor_1_ax,sensor_1_ay,sensor_1_az,sensor_1_gx,sensor_1_gy,sensor_1_gz,sensor_2_ax,sensor_2_ay,sensor_2_az,sensor_2_gx,sensor_2_gy,sensor_2_gz,onboard_ax,onboard_ay,onboard_az,onboard_gx,onboard_gy,onboard_gz
0,1,1,-0.099558,-0.662749,0.025841,0.365632,-0.578672,-0.851637,-0.159671,-0.446847,0.428461,-0.655290,-0.399092,-0.770198,-0.674266,0.518070,-0.948649,0.068775,0.484752,0.211172
1,1,1,-0.722363,0.771889,-0.049348,-0.336130,-0.565412,0.400601,-0.038406,0.035937,0.854382,0.433370,-0.147353,-0.069629,0.660145,-0.866939,0.853632,-0.221816,-0.759564,-0.659396
2,1,1,-0.415701,-0.021842,-0.334597,-0.574242,-0.341228,-0.556007,-0.638761,0.864486,0.633500,-0.261101,0.155902,-0.256496,0.785208,0.773856,0.345934,-0.043762,0.437023,-0.696030
3,1,1,0.465464,-0.260321,-0.390624,-0.137906,0.522348,0.861324,-0.729307,0.159677,-0.327382,0.267208,0.890500,0.375132,-0.906876,0.923590,0.541872,0.512648,-0.407945,0.072166
4,1,1,-0.077897,0.503816,0.890514,-0.186465,-0.814148,-0.334972,-0.014068,-0.853282,-0.803982,0.387693,-0.262085,0.342637,-0.643610,0.951537,0.370544,0.570446,0.890018,-0.086719


In [24]:
# Gesture code to gesture label dict
with open('raw_data_collection/dummy_gesture_dict.json', 'r') as json_file:
    gesture_dict = json.load(json_file)
gesture_dict


{'gesture_1': 1,
 'gesture_2': 2,
 'gesture_3': 3,
 'gesture_4': 4,
 'gesture_5': 5}

## EDA

In [25]:
# let's create a groupby generator for future use
visual_group_by = raw_sensor_data.groupby('gesture_code')

# let's see what the data looks like.
print('data description')
display(raw_sensor_data.describe())

# let's observe data for one gesture
print('\n')
for gesture_code, gesture_df in visual_group_by:
    print('gesture_code: ',gesture_code)
    display(gesture_df)
    break

data description


,iteration,gesture_code,sensor_1_ax,sensor_1_ay,sensor_1_az,sensor_1_gx,sensor_1_gy,sensor_1_gz,sensor_2_ax,sensor_2_ay,sensor_2_az,sensor_2_gx,sensor_2_gy,sensor_2_gz,onboard_ax,onboard_ay,onboard_az,onboard_gx,onboard_gy,onboard_gz
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,50.500000,3.000000,-0.006395,-0.000454,-0.003486,-0.003894,0.008021,0.000956,-0.006119,0.000126,-0.007028,0.000503,0.003680,0.008513,0.005134,-0.000241,-0.001524,0.004370,0.003168,0.002442
std,28.867032,1.414261,0.578387,0.576301,0.578094,0.573520,0.576648,0.581218,0.580413,0.575462,0.582078,0.574622,0.578675,0.573913,0.580507,0.577176,0.576181,0.577187,0.577463,0.578988
min,1.000000,1.000000,-0.999963,-0.999972,-0.999652,-0.999950,-0.999973,-0.999970,-0.999993,-0.999981,-0.999881,-0.999895,-0.999966,-0.999894,-0.999920,-0.999937,-0.999920,-0.999944,-0.999916,-0.999670
25%,25.750000,2.000000,-0.508832,-0.497244,-0.507781,-0.500694,-0.490817,-0.503303,-0.508680,-0.500141,-0.515899,-0.494098,-0.498003,-0.478397,-0.501309,-0.496873,-0.505746,-0.498222,-0.502389,-0.497951
50%,50.500000,3.000000,-0.009009,-0.007044,-0.009860,-0.004931,0.018281,-0.003004,-0.010767,0.008245,-0.004018,-0.004473,0.001451,0.014453,0.007281,0.002556,-0.002538,0.013036,0.005101,0.015712
75%,75.250000,4.000000,0.495997,0.495306,0.500767,0.491624,0.508055,0.513993,0.494218,0.494328,0.500186,0.499769,0.507168,0.501388,0.509519,0.497394,0.495837,0.497167,0.500719,0.502771
max,100.000000,5.000000,0.999885,0.999962,0.999991,0.999818,0.999995,0.999367,0.999953,0.999976,0.999946,0.999929,0.999990,0.999776,0.999921,0.999851,0.999531,1.000000,0.999934,0.999976




gesture_code:  1


,iteration,gesture_code,sensor_1_ax,sensor_1_ay,sensor_1_az,sensor_1_gx,sensor_1_gy,sensor_1_gz,sensor_2_ax,sensor_2_ay,sensor_2_az,sensor_2_gx,sensor_2_gy,sensor_2_gz,onboard_ax,onboard_ay,onboard_az,onboard_gx,onboard_gy,onboard_gz
0,1,1,-0.099558,-0.662749,0.025841,0.365632,-0.578672,-0.851637,-0.159671,-0.446847,0.428461,-0.655290,-0.399092,-0.770198,-0.674266,0.518070,-0.948649,0.068775,0.484752,0.211172
1,1,1,-0.722363,0.771889,-0.049348,-0.336130,-0.565412,0.400601,-0.038406,0.035937,0.854382,0.433370,-0.147353,-0.069629,0.660145,-0.866939,0.853632,-0.221816,-0.759564,-0.659396
2,1,1,-0.415701,-0.021842,-0.334597,-0.574242,-0.341228,-0.556007,-0.638761,0.864486,0.633500,-0.261101,0.155902,-0.256496,0.785208,0.773856,0.345934,-0.043762,0.437023,-0.696030
3,1,1,0.465464,-0.260321,-0.390624,-0.137906,0.522348,0.861324,-0.729307,0.159677,-0.327382,0.267208,0.890500,0.375132,-0.906876,0.923590,0.541872,0.512648,-0.407945,0.072166
4,1,1,-0.077897,0.503816,0.890514,-0.186465,-0.814148,-0.334972,-0.014068,-0.853282,-0.803982,0.387693,-0.262085,0.342637,-0.643610,0.951537,0.370544,0.570446,0.890018,-0.086719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,100,1,-0.908817,-0.991828,-0.441631,0.981450,-0.609583,0.007075,0.472649,-0.430702,-0.011327,-0.266702,-0.585783,0.091386,-0.349047,0.759284,0.570033,-0.054457,-0.432265,-0.897375
2996,100,1,-0.456429,-0.208215,0.479586,-0.911659,0.923212,-0.598330,0.362397,-0.466529,-0.105874,0.296621,-0.684187,0.751722,0.414960,0.200094,0.247316,-0.484533,-0.986094,0.098585
2997,100,1,-0.006708,0.669189,-0.409789,0.932738,0.901336,0.356733,-0.211302,-0.372371,-0.944026,0.168318,0.822070,0.764565,0.484923,0.301645,-0.960188,0.164838,0.961459,-0.542556
2998,100,1,-0.375877,-0.425982,0.308036,0.608524,0.465420,-0.415138,0.929686,0.831164,-0.206788,-0.257876,0.212709,0.684137,0.730963,0.474223,-0.612372,0.910510,0.559009,-0.875796


### lets visualize sensor data for each gesture

In [26]:
# building one altair chart
def build_a_viz(raw_table,x_column_name,y_column_name):
    one_chart = alt.Chart(raw_table).mark_circle().encode(
        x=x_column_name,
        y=y_column_name
    )
    return one_chart

# combining multiple visualizations together
def build_multi_viz(gesture_table):
    gesture_table.drop(['iteration','gesture_code'],axis=1,inplace = True)
    sensor_1_chart_list = []
    sensor_2_chart_list = []
    onboard_chart_list = []
    for a_column in gesture_table.columns:
        one_chart = build_a_viz(gesture_table,'sequence_position',a_column)
        if 'sensor_1' in a_column:
            sensor_1_chart_list.append(one_chart)
        if 'sensor_2' in a_column:
            sensor_2_chart_list.append(one_chart)
        if 'onboard' in a_column:
            onboard_chart_list.append(one_chart)
        
    return alt.vconcat(*[alt.hconcat(*sensor_1_chart_list),alt.hconcat(*sensor_2_chart_list),alt.hconcat(*onboard_chart_list)])
        

In [27]:
plot_dict = {}
for gesture, gesture_df in visual_group_by:
    # lets create a repeating sequence from [0,30) for every gesture and iteration
    reset_df = gesture_df.reset_index().astype(float)
    
    # reading_count is defined in the first block of this notebook
    reset_df['sequence_position'] = (reset_df['index'])%reading_count
    plot = build_multi_viz(reset_df)
    
    # let's plot 
    for gesture_name,gesture_code in gesture_dict.items():
        if gesture_code == gesture:
            plot_dict[gesture_name] = plot

plot_dict

{'gesture_1': alt.VConcatChart(...),
 'gesture_2': alt.VConcatChart(...),
 'gesture_3': alt.VConcatChart(...),
 'gesture_4': alt.VConcatChart(...),
 'gesture_5': alt.VConcatChart(...)}

In [28]:
# letssee one of these plots
plot_dict['gesture_1']

/opt/homebrew/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

## Feature Development

We are creating features that describe the distribution of the sensor data for every iteration of every gesture

In [41]:
# the features we are interested in:
def build_features(series):
    mean_value = series.mean()
    std_deviation = series.std()
    skewness = series.skew()
    kurtosis_value = series.kurtosis()
    minimum = min(series)
    maximum = max(series)
    return mean_value, std_deviation, skewness, kurtosis_value, minimum, maximum

def feature_pipeline(output_dict,one_gesture_df,key_columns, gesture_key):
    features_list = ['mean_value', 'std_deviation', 'skewness', 'kurtosis_value', 'minimum', 'maximum']

    if len(output_dict) == 0:
        for sensor_column in key_columns:
            for a_feature in features_list:
                output_dict[sensor_column + '_' + a_feature] = []
        output_dict['gesture_label'] = []


    for sensor_column in key_columns:
        mean_value, std_deviation, skewness, kurtosis_value, minimum, maximum = build_features(one_gesture_df[sensor_column])
        output_dict[sensor_column + '_' + 'mean_value'].append(mean_value)
        output_dict[sensor_column + '_' + 'std_deviation'].append(std_deviation)
        output_dict[sensor_column + '_' + 'skewness'].append(skewness)
        output_dict[sensor_column + '_' + 'kurtosis_value'].append(kurtosis_value)
        output_dict[sensor_column + '_' + 'minimum'].append(minimum)
        output_dict[sensor_column + '_' + 'maximum'].append(maximum)
    output_dict['gesture_label'].append(gesture_key)
    return output_dict

In [42]:
raw_data_columns = ['sensor_1_ax','sensor_1_ay','sensor_1_az','sensor_1_gx','sensor_1_gy','sensor_1_gz','sensor_2_ax','sensor_2_ay','sensor_2_az','sensor_2_gx','sensor_2_gy','sensor_2_gz','onboard_ax','onboard_ay','onboard_az','onboard_gx','onboard_gy','onboard_gz']
output_dict = {}
for (gesture_code,gesture_iteration), gesture_df in raw_sensor_data.groupby(['gesture_code','iteration']):
    output_dict = feature_pipeline(output_dict,gesture_df,raw_data_columns, gesture_code)

feature_df = pd.DataFrame(output_dict)
feature_df

,sensor_1_ax_mean_value,sensor_1_ax_std_deviation,sensor_1_ax_skewness,sensor_1_ax_kurtosis_value,sensor_1_ax_minimum,sensor_1_ax_maximum,sensor_1_ay_mean_value,sensor_1_ay_std_deviation,sensor_1_ay_skewness,sensor_1_ay_kurtosis_value,...,onboard_gy_kurtosis_value,onboard_gy_minimum,onboard_gy_maximum,onboard_gz_mean_value,onboard_gz_std_deviation,onboard_gz_skewness,onboard_gz_kurtosis_value,onboard_gz_minimum,onboard_gz_maximum,gesture_label
0,0.019834,0.565836,-0.048767,-1.043838,-0.945619,0.910030,0.023818,0.518744,-0.094933,-0.846173,...,-1.457405,-0.964426,0.973764,-0.086674,0.569228,0.252855,-0.917884,-0.987765,0.934237,1
1,-0.020847,0.501358,0.053056,-0.820406,-0.902058,0.938843,0.166980,0.609910,-0.342812,-1.055357,...,-1.088154,-0.984655,0.958038,0.049598,0.521378,0.126695,-0.991638,-0.790073,0.966735,1
2,-0.198860,0.542655,0.174377,-0.681539,-0.998883,0.941036,0.068918,0.564650,-0.313217,-0.903567,...,-1.069175,-0.999730,0.784961,0.003525,0.585266,-0.266358,-1.171928,-0.971113,0.915810,1
3,-0.103432,0.527995,-0.077653,-1.025747,-0.959666,0.786324,0.054760,0.620297,-0.087898,-1.185769,...,-0.513711,-0.920753,0.978529,0.012358,0.543868,-0.238637,-1.181067,-0.923729,0.822018,1
4,0.049231,0.646059,-0.260579,-1.299963,-0.984003,0.988430,0.032818,0.618438,-0.105524,-1.499043,...,-0.668418,-0.984307,0.978456,0.022013,0.530590,-0.045543,-0.989253,-0.857080,0.930836,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.035914,0.620626,-0.160348,-1.244004,-0.981031,0.920593,-0.110547,0.592952,0.262449,-1.023874,...,-0.962283,-0.978902,0.995355,-0.072095,0.671207,0.104953,-1.697377,-0.954418,0.940709,5
496,-0.060020,0.616290,0.307950,-1.073464,-0.965020,0.979064,-0.051491,0.609706,0.021288,-1.366412,...,-1.579653,-0.954918,0.992122,-0.094249,0.623333,0.340538,-1.207283,-0.990552,0.970777,5
497,-0.095309,0.548377,0.195538,-0.786905,-0.967175,0.909613,-0.183649,0.509096,0.368857,-0.843780,...,-0.977681,-0.991096,0.977621,0.176966,0.626159,-0.478261,-1.183180,-0.968739,0.972899,5
498,0.004808,0.568122,-0.022593,-1.322631,-0.954887,0.945316,-0.013883,0.537652,0.364991,-1.306926,...,-0.631839,-0.963017,0.916285,-0.063699,0.661103,0.325815,-1.378650,-0.984228,0.997088,5


## Train, Test Split